# Dependencias y conexión a mongo

In [ ]:
!pip install pymongo[srv]

In [1]:
# Revisar las dependencias y si la colección esta creada
from pymongo import MongoClient
from bson import json_util
import json


MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017
DB_NAME = 'OCDS'
COLLECTION_NAME = 'Contracts'

connection = MongoClient(MONGODB_HOST, MONGODB_PORT)

In [6]:
db = connection.OCDS
collection = db['Contracts']

In [11]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'OCDS'), 'Contracts')

In [17]:
# Query para ver contratos cargados en la colección Contracts
result = db.Contracts.aggregate([
    {
        '$count' : 'uri'
    }
])

print(list(result))

[{'uri': 1874635}]


# Ingresar json a mongo

In [2]:
import glob

# Seleccionar archivos de un mes en particular
LIC = glob.glob("./Data/LIC/JSON/202108/*")
TD = glob.glob("./Data/TD/JSON/202108/*")

In [3]:
print(len(LIC))
print(len(TD))

12376
62654


In [7]:
e = 0
f = 0
for file in TD:
    file_json = open(file, encoding="utf8")
    try:
        file_data = json.load(file_json)
        try:
            a = collection.insert_one(file_data)
        except:
            e = e + 1
    except:
        f = f + 1
    file_json.close()

print('Errores en archivos json:', f, '\nErrores al cargar en mongo:', e)

Errores en archivos json: 0 
Errores al cargar en mongo: 0


In [ ]:
e = 0
f = 0
for file in LIC:
    file_json = open(file, encoding="utf8")
    try:
        file_data = json.load(file_json)
        try:
            a = collection.insert_one(file_data)
        except:
            e = e + 1
    except:
        f = f + 1
    file_json.close()

print('Errores en archivos json:', f, '\nErrores al cargar en mongo:', e)

# Procesar los datos en mongo

In [8]:
# Conjunto de contratos (licitaciones)
client = MongoClient('mongodb://localhost:27017/')
result = client['OCDS']['Contracts'].aggregate([
    {
        '$match': {
            '$and': [
                {
                    'status': {
                        '$ne': 500
                    }
                }, {
                    'status': {
                        '$ne': 404
                    }
                }
            ]
        }
    }, {
        '$match': {
            'records': {
                '$exists': True
            }
        }
    }, {
        '$project': {
            'records': 1, 
            'uri': 1
        }
    }, {
        '$addFields': {
            'records': {
                '$last': '$records'
            }
        }
    }, {
        '$addFields': {
            'Aregistros': {
                '$last': '$records.releases'
            }, 
            'Tregistros': {
                '$first': '$records.releases'
            }, 
            'ocid': '$records.ocid'
        }
    }, {
        '$addFields': {
            'Aregistro': {
                '$last': '$Aregistros.awards'
            }, 
            'Tregistro': '$Tregistros.tender'
        }
    }, {
        '$project': {
            'records': 0, 
            'compiladoRegistros': 0, 
            'Aregistros': 0, 
            'Tregistros': 0, 
            'releases': 0
        }
    }, {
        '$addFields': {
            'title': '$Tregistro.title', 
            'Testado': '$Tregistro.status', 
            'Aestado': '$Aregistro.status', 
            'Tipo': '$Tregistro.procurementMethod', 
            'TipoDetalle': '$Tregistro.procurementMethodDetails'
        }
    }, {
        '$addFields': {
            'TenderDateStart': {
                '$toDate': '$Tregistro.tenderPeriod.startDate'
            }, 
            'TenderDateEnd': {
                '$toDate': '$Tregistro.tenderPeriod.endDate'
            }, 
            'AwardDate': {
                '$toDate': '$Aregistro.date'
            }, 
            'TenderValue': '$Tregistro.value.amount', 
            'TenderCurrency': '$Tregistro.value.currency', 
            'AwardValue': '$Aregistro.value.amount', 
            'AwardCurrency': '$Aregistro.value.currency'
        }
    }, {
        '$project': {
            '_id': 1, 
            'uri': 1, 
            'ocid': 1, 
            'title': 1, 
            'Testado': 1, 
            'Aestado': 1, 
            'Tipo': 1, 
            'TipoDetalle': 1, 
            'Tipo2': 1, 
            'TenderDateStart': 1, 
            'TenderDateEnd': 1, 
            'AwardDate': 1, 
            'TenderValue': 1, 
            'TenderCurrency': 1, 
            'AwardValue': 1, 
            'AwardCurrency': 1
        }
    }, {
        '$out': 'OCDS-LIC-F'
    }
])

In [9]:
# Conjunto de contratos (tratos directos)
client = MongoClient('mongodb://localhost:27017/')
result = client['OCDS']['Contracts'].aggregate([
    {
        '$match': {
            '$and': [
                {
                    'status': {
                        '$ne': 500
                    }
                }, {
                    'status': {
                        '$ne': 404
                    }
                }
            ]
        }
    }, {
        '$match': {
            'releases': {
                '$exists': True
            }
        }
    }, {
        '$project': {
            'releases': 1, 
            'uri': 1
        }
    }, {
        '$addFields': {
            'records': {
                '$last': '$releases'
            }
        }
    }, {
        '$addFields': {
            'ocid': '$records.ocid', 
            'Tregistros': '$records.tender', 
            'Aregistros': '$records.awards', 
            'TenderDateStart': None, 
            'TenderDateEnd': None
        }
    }, {
        '$unwind': {
            'path': '$Aregistros'
        }
    }, {
        '$project': {
            'records': 0, 
            'compiladoRegistros': 0, 
            'releases': 0
        }
    }, {
        '$addFields': {
            'title': '$Aregistros.title', 
            'Testado': 'Trato directo', 
            'Aestado': '$Aregistros.status', 
            'Tipo': '$Tregistros.procurementMethod', 
            'TipoDetalle': 'Trato Directo'
        }
    }, {
        '$addFields': {
            'AwardDate': {
                '$toDate': '$Aregistros.date'
            }, 
            'TenderValue': None, 
            'TenderCurrency': None, 
            'AwardValue': '$Aregistros.value.amount', 
            'AwardCurrency': '$Aregistros.value.currency'
        }
    }, {
        '$project': {
            'Tregistros': 0, 
            'Aregistros': 0
        }
    }, {
        '$out': 'OCDS-LIC-TD-F'
    }
])

In [10]:
# Conjunto de actores en licitaciones
client = MongoClient('mongodb://localhost:27017/')
result = client['OCDS']['Contracts'].aggregate([
    {
        '$match': {
            'records.releases': {
                '$exists': True
            }
        }
    }, {
        '$addFields': {
            'releases': {
                '$last': '$records.releases'
            }
        }
    }, {
        '$addFields': {
            'releases': {
                '$last': '$releases'
            }
        }
    }, {
        '$addFields': {
            'parties': '$releases.parties'
        }
    }, {
        '$project': {
            '_id': 1, 
            'parties': 1, 
            'parties2': 1
        }
    }, {
        '$unwind': {
            'path': '$parties'
        }
    }, {
        '$unwind': {
            'path': '$parties.roles'
        }
    }, {
        '$addFields': {
            'name': '$parties.name', 
            'id': '$parties.id', 
            'region': '$parties.address.region', 
            'rol': '$parties.roles', 
            'name2': '$parties.identifier.legalName', 
            'id2': '$_id'
        }
    }, {
        '$project': {
            'parties': 0, 
            '_id': 0
        }
    }, {
        '$match': {
            '$or': [
                {
                    'rol': 'buyer'
                }, {
                    'rol': 'supplier'
                }
            ]
        }
    }, {
        '$out': 'OCDS-PAR-LIC-F'
    }
])

In [11]:
# Conjunto de actores en tratos directos
client = MongoClient('mongodb://localhost:27017/')
result = client['OCDS']['Contracts'].aggregate([
    {
        '$match': {
            'releases': {
                '$exists': True
            }
        }
    }, {
        '$addFields': {
            'releases': {
                '$last': '$releases'
            }
        }
    }, {
        '$addFields': {
            'parties': '$releases.parties'
        }
    }, {
        '$project': {
            '_id': 1, 
            'parties': 1
        }
    }, {
        '$unwind': {
            'path': '$parties'
        }
    }, {
        '$unwind': {
            'path': '$parties.roles'
        }
    }, {
        '$addFields': {
            'name': '$parties.name', 
            'id': '$parties.id', 
            'region': '$parties.address.region', 
            'rol': '$parties.roles', 
            'name2': '$parties.identifier.legalName', 
            'id2': '$_id'
        }
    }, {
        '$project': {
            'parties': 0, 
            '_id': 0
        }
    }, {
        '$match': {
            '$or': [
                {
                    'rol': 'buyer'
                }, {
                    'rol': 'supplier'
                }
            ]
        }
    }, {
        '$out': 'OCDS-PAR-TD-F'
    }
])

# Extraer los datos procesados en mongo

In [12]:
# Conjunto de licitaciones

import pandas as pd
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
result = client['OCDS']['OCDS-LIC-F'].aggregate([
    {
        '$match': {}
    }
])
LIC = pd.DataFrame(list(result))
LIC.to_csv('./CONS/LIC.csv', index=False)
del LIC

# Conjunto de participantes en la licitación
result = client['OCDS']['OCDS-PAR-LIC-F'].aggregate([
    {
        '$match': {}
    }
])
PAR = pd.DataFrame(list(result))
PAR.to_csv('./CONS/PAR-LIC.csv', index=False)
del PAR

In [13]:
# Conjunto de tratos directos

import pandas as pd
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
result = client['OCDS']['OCDS-LIC-TD-F'].aggregate([
    {
        '$match': {}
    }
])
LIC = pd.DataFrame(list(result))
LIC.to_csv('./CONS/LIC-TD.csv', index=False)
del LIC

# Conjunto de participantes en los tratos directos
result = client['OCDS']['OCDS-PAR-TD-F'].aggregate([
    {
        '$match': {}
    }
])
TC = pd.DataFrame(list(result))
TC.to_csv('./CONS/PAR-TD.csv', index=False)
del TC